# xG Model

In [4]:
from statsbombpy import sb
import matplotlib.pyplot as plt
from mplsoccer.pitch import Pitch
import seaborn as sns
import json
import numpy as np
import pandas as pd
import networkx as nx
import math

In [6]:
with open(r'C:\Users\34626\OneDrive\Documents\Jaume\Big Data\Football\events/events_Spain.json') as f:
    data = json.load(f)

In [13]:
#Create a data set of shots.
train = pd.DataFrame(data)
pd.unique(train['subEventName'])
shots = train[train['subEventName'] == 'Shot' ]
shots_model = pd.DataFrame(columns=['Goal', 'X', 'Y'])

In [15]:
#Go through the dataframe and calculate X, Y co-ordinates.
#Distance from a line in the centre
#Shot angle.
#Details of tags can be found here: https://apidocs.wyscout.com/matches-wyid-events

In [16]:
for i, shot in shots.iterrows():

    header=0
    for shottags in shot['tags']:
        if shottags['id'] == 403:
            header=1
    #Only include non-headers          
    if not(header):
        shots_model.at[i,'X']=100-shot['positions'][0]['x']
        shots_model.at[i,'Y']=shot['positions'][0]['y']
        shots_model.at[i,'C']=abs(shot['positions'][0]['y']-50) 

        #Distance in metres and shot angle in radians.
        x=shots_model.at[i,'X']*105/100
        y=shots_model.at[i,'C']*65/100
        shots_model.at[i,'Distance']=np.sqrt(x**2 + y**2)
        a = np.arctan(7.32 *x /(x**2 + y**2 - (7.32/2)**2))

        if a<0:
            a=np.pi+a
        shots_model.at[i, 'Angle'] = a

        #Was it a goal
        shots_model.at[i,'Goal']=0
        for shottags in shot['tags']:
                #Tags contain that its a goal
                if shottags['id']==101:
                    shots_model.at[i,'Goal']=1

In [17]:
shots_model

,Goal,X,Y,C,Distance,Angle
20,0,7,34,16.0,12.735089,0.346975
22,0,20,59,9.0,21.799599,0.321317
107,0,12,57,7.0,13.396361,0.506972
111,0,13,66,16.0,17.160492,0.341540
228,0,25,40,10.0,27.042790,0.261520
...,...,...,...,...,...,...
628323,0,7,66,16.0,12.735089,0.346975
628374,0,10,32,18.0,15.720687,0.317684
628540,1,8,25,25.0,18.292690,0.189128
628596,0,18,30,20.0,22.939268,0.263508


In [18]:
#Two dimensional histogram
H_Shot=np.histogram2d(shots_model['X'], shots_model['Y'],bins=50,range=[[0, 100],[0, 100]])
goals_only=shots_model[shots_model['Goal']==1]
H_Goal=np.histogram2d(goals_only['X'], goals_only['Y'],bins=50,range=[[0, 100],[0, 100]])